# graph see 2

# Preliminaries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from more_itertools import collapse, flatten, groupby_transform, unique_justseen
import numpy as np
from operator import itemgetter
from pprint import pprint
import neo4j
import tools.neotools as nj

In [3]:
import ipywidgets as widgets
from sidecar import Sidecar
from nnvis import ADCResponsePlot

In [4]:
from graph_utils_neo4j import NumpyStore
import grexutils as gu
from nnbench import Thing

## Connecting

In [5]:
driver = neo4j.GraphDatabase.driver("neo4j://neo4j:7687", auth=("neo4j", "test"))
driver.verify_connectivity()

<ipython-input-5-e75ba85668dd>:2: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


{IPv4Address(('neo4j', 7687)): [{'ttl': 300,
   'servers': [{'addresses': ['neo4j:7687'], 'role': 'WRITE'},
    {'addresses': ['neo4j:7687'], 'role': 'READ'},
    {'addresses': ['neo4j:7687'], 'role': 'ROUTE'}]}]}

# The Model
    Investigation -> Experiment -> multiple ResultDAGs
`ResultDAG` is

    (netState, params)-[mutation]->(netState, params)-[mutation ...
                     +-[mutation]->(netstate, params) ...
etc. `mutation` can be a learning trajectory, or an edit.

Perhaps `mutation` can be expressed in python.

Generally the results of experiments are preferred to be reproducible, but they won't always be, when they import entropy.

## Some neural nets

In [6]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from nnbench import NetMaker, NNMEG

In [7]:
mnm = NetMaker(NNMEG)
xor_net = mnm('2x2tx1t')
adc_net = mnm('1x8tx8tx3t')

## ... and training data

In [8]:
xor_training_batch = (np.array([[-0.5, -0.5],
                            [-0.5,  0.5],
                            [ 0.5,  0.5],
                            [ 0.5, -0.5]]),
                  np.array([[-0.5],
                            [ 0.5],
                            [-0.5],
                            [ 0.5]]))

In [9]:
def adc(input):
    m = max(0, min(7, int(8*input)))
    return np.array([(m>>2)&1, (m>>1)&1, m&1]) * 2 - 1

vadc = lambda v: np.array([adc(p) for p in v])
#plot_ADC(vadc)

In [10]:
x = np.arange(0, 1, 1.0/(8*1)).reshape(-1,1) # 1 point in each output region
adc_training_batch = (x, vadc(x))

# Navigate the data

In [11]:
class SMChangeFilter():
    def __init__(self, w):
        self.w = w
        self.val = w.value
        #self.counter = 0
        
    def __call__(self, cb):
        self.w.observe(self._on_change, names='value')
        self.cb = cb
        
    def _on_change(self, d):
        #self.counter += 1
        #print(f"SMChangeFilter count={self.counter}")
        new_val = d['new']
        # De-noise
        if len(set(new_val)) == len(new_val):
            self.cb(new_val)

In [20]:
class ADCExperimentResultsSelector():
    def __init__(self, driver):
        self.driver = driver
        
        exp_w = widgets.Dropdown(
            options=sorted(gu.get_experiment_names_keys(driver)),
            description='Experiment:',
        )

        proc_w = widgets.Dropdown(
            options = sorted(gu.get_procedure_names_keys_from_experiment_key(driver, exp_w.value)),
            description='Procedure:',
        )

        params_w_starting_options = sorted(gu.parameter_names_keys_from_experiment_and_procedure_keys(driver, exp_w.value, proc_w.value))
        params_w = widgets.SelectMultiple(
            options=params_w_starting_options,
            #value=params_w_starting_options[0], # FIXME: shouldn't this work?
            rows=16,
            description='Parameters',
        )

        def on_exp_w_change(d):
            proc_w.options = gu.get_procedure_names_keys_from_experiment_key(driver, d['new'])

        exp_w.observe(on_exp_w_change, names='value')

        def on_proc_w_change(d):
            self.params_w.options = \
                sorted(gu.started_parameter_names_keys_from_experiment_and_procedure_keys(driver, exp_w.value,  d['new']))

        proc_w.observe(on_proc_w_change, names='value')
        
        #self.params_w.observe(on_params_w_change, names='value')
        params_cf = SMChangeFilter(params_w)

        self.params_cf = params_cf
        self.exp_w, self.proc_w, self.params_w = exp_w, proc_w, params_w

    def __call__(self, cb):
        self.params_cf(cb)

    def get_widgets(self):
        return self.exp_w, self.proc_w, self.params_w

    def _repr_html_(self):
        # I don't know how to do this
        # https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html
        return display(self.get_widgets(), include=('text/html'))

In [13]:
nps = NumpyStore(driver)

In [14]:
aers = ADCExperimentResultsSelector(driver)

In [15]:
class PrintEnumerated():
    def __init__(self, out):
        self.out = out
        self.ctr = 0
        
    def __call__(self, v):
        self.ctr += 1
        with self.out:
            print(f"{self.ctr}: {v}")

In [16]:
output2 = widgets.Output()
aers(PrintEnumerated(output2))

In [17]:
display(*aers.get_widgets(), output2)

Dropdown(description='Experiment:', options=(('t3', '7R42cwhPZvhpmEUd51W5xQ'), ('t4', 'jtsgosW1bfzg3SdgkYLqSw'…

Dropdown(description='Procedure:', options=(('Train ADCs', 'j-xrJdlnO71OTXjar2Pe3w'), ('more Train ADCs', 'ryD…

SelectMultiple(description='Parameters', options=(('go easy 0', '8iNtNDfhgJhYpF7o-Ied2w'), ('go easy 0', 'GPAC…

Output()

In [21]:
aers.params_w.rows=16

In [25]:
exp_results = Thing()

In [26]:
def update_exp_results(parameter_keys):
    exp_results.params_keys = parameter_keys
    exp_results.params = [gu.Parameters(driver, nps, key) for key in parameter_keys]

In [27]:
aers(update_exp_results)

In [28]:
def postprocess_trajectory(traj):
    if max(traj.etas) > len(traj.etas):
        # This one we assume is old and has change_batches swapped with etas. Swap to correct:
        t = traj.eta_change_batches
        traj.eta_change_batches = traj.etas
        traj.etas = t
    traj.batches_etas = list(unique_justseen(zip(traj.eta_change_batches, traj.etas), itemgetter(1)))
    return traj

In [32]:
exp_results.params[0].name, exp_results.params[0].unikey

('go easy 108', 'JJ4KYOVDd0JdaB1ckmweIQ')

In [33]:
t = postprocess_trajectory(exp_results.params[0].trajectory)

In [34]:
t.batches_etas

[(0, 0.1)]

In [35]:
from tools.stripchart import Losschart

In [36]:
loss_chart = Losschart(1000, height='220px')
loss_chart.fig

Figure(axes=[Axis(label='batch', num_ticks=4, scale=LinearScale()), Axis(label='loss', num_ticks=4, orientatio…

In [37]:
loss_chart((t.batch_points, t.losses))

In [51]:
nrps = [ADCResponsePlot(resolution=1/96, height='220px', margin=30, title=p.name)
        for p in exp_results.params]

In [52]:
loss_charts = [Losschart(1_000, height='220px', margin=30, title=p.name)
        for p in exp_results.params]

In [53]:
_ = [lc((p.trajectory.batch_points, p.trajectory.losses)) for lc, p in zip(loss_charts, exp_results.params)]

In [54]:
def show_adcs_n(n):
    for nrp, par in zip(nrps, exp_results.params):
        try:
            res = par.results[n]
        except IndexError:
            res = par.results[-1]
        nrp(res.nnet)
        nrp.fig.title = f"{par.name} {res.loss:.3f}"

In [55]:
frame_w = widgets.IntSlider(min=0,
                            max=max(len(p.results) for p in exp_results.params)-1,
                            step=1, value=0)

# Skip the grid
plots_box = widgets.Box(tuple(flatten(zip((nrp.fig for nrp in nrps), (lc.fig for lc in loss_charts)))),
                        layout=widgets.Layout(flex_flow='row wrap',
                                             justify_content='space-around'))
        
with Sidecar(title='ADCs') as side:
    display(plots_box)

In [49]:
widgets.interact(show_adcs_n, n=frame_w)

interactive(children=(IntSlider(value=0, description='n', max=69), Output()), _dom_classes=('widget-interact',…

<function __main__.show_adcs_n(n)>

In [ ]:
assert False, "Pause here"

___

# Scrapyard

In [ ]:
assert False, "Scrapyard below"

In [ ]:
from functools import lru_cache as cache
from collections import defaultdict

In [ ]:
import re
import time

In [ ]:
nrps[-1].fig.save_png('go easy 131.png')

In [ ]:
nrps[-1].fig.save_svg('go easy 131.svg')

In [ ]:
nps['341ffbe2800325e7']

In [ ]:
time.gmtime(0)

In [ ]:
time.ctime(1615778080.564)

In [ ]:
time.time()

In [ ]:
exp_w.value

In [ ]:
proc_w.options = gu.get_procedure_names_keys_from_experiment_key(driver, exp_w.value)

In [ ]:
proc_w.options

In [ ]:
params_w.options = sorted(gu.parameter_names_keys_from_experiment_and_procedure_keys(driver, exp_w.value, proc_w.value))

In [ ]:
def sorted_parameter_names_keys_from_experiment_and_procedure_keys(*args):
    import re
    def atoi(text):
        return int(text) if text.isdigit() else text
    
    def natural_keys(text):
        return [ atoi(c) for c in re.split('(\d+)',text) ]
    
    my_list =['Hello1', 'Hello12', 'Hello29', 'Hello2', 'Hello17', 'Hello25']
    my_list =['Hello 1', 'Hello 12', 'Hello 29', 'Hello 2', 'Hello 17', 'Hello 25']
    my_list.sort(key=natural_keys)
    return my_list
sorted_parameter_names_keys_from_experiment_and_procedure_keys()

In [ ]:
def sorted_parameter_names_keys_from_experiment_and_procedure_keys(*args):
    vl = gu.parameter_names_keys_from_experiment_and_procedure_keys(*args)
    vl = [v[0] for v in gu.parameter_names_keys_from_experiment_and_procedure_keys(*args)]
    vl.sort(key=lambda t:[(lambda s: s.isdigit() and int(s) or s)(v) for v in re.split('(\d+)', t[0])])
    return vl
sorted_parameter_names_keys_from_experiment_and_procedure_keys(driver, exp_w.value, proc_w.value)

In [ ]:
%debug

In [ ]:
par = gu.Parameters(driver, nps, 'bZQGcrdr1-kqf1ADGZNQJw')

In [ ]:
par.results[:3]

In [ ]:
class PrintEnumerated():
    def __init__(self, out):
        self.out = out
        self.ctr = 0
        
    def __call__(self, v):
        self.ctr += 1
        with self.out:
            print(f"{self.ctr}: {v}")

In [ ]:
output2 = widgets.Output()
aers(PrintEnumerated(output2))

In [ ]:
exp_results.params_keys, exp_results.params

In [ ]:
exp_results.params[0].results

In [ ]:
par.name

In [ ]:
net = par.results[0].nnet
net(np.array([0.7]))

In [ ]:
exp_results.params[0].results[0].nnet(np.array([0.7])) 

In [ ]:
#ADCResponsePlot(title=par.name)(net).fig

In [ ]:
nets = [mnm(t.shorthand) for t in things]
nrps = [NetResponsePlot(net, height='220px', margin=30, title=thing.name)
        for net, thing in zip(nets, things)]

frame_w = widgets.IntSlider(min=0,
                            max=max(len(t.ksvs) for t in things)-1,
                            step=1, value=0)

# Skip the grid
plots_box = widgets.Box(tuple(nrp.fig for nrp in nrps),
                        layout=widgets.Layout(flex_flow='row wrap',
                                             justify_content='space-around'))
        
with Sidecar(title='grid') as gside:
    display(plots_box)
    
widgets.interact(show_adcs_n, n=frame_w)

In [ ]:
param = gu.Parameters(driver, 'T33KUugDQrnou2fjKRFHFg')

In [ ]:
param.name

In [ ]:
def was_data_from_run():
    def get_sequence(tx, rv):
        for record in tx.run("MATCH p=(head:Net)-[:LEARNED*]->(tail:Net) "
                             "WHERE NOT ()-[:LEARNED]->(head) "
                             "AND NOT (tail)-[:LEARNED]->() "
                             "RETURN "
                             "head, "
                             "[x IN nodes(p) | x.ksv] as ksvs, "
                             "[x IN nodes(p) | x.loss] as losses "
                            ):
            rv.head = record['head']
            rv.ksvs = record['ksvs']
            rv.losses = record['losses']

    rv = Thing
    with driver.session() as session:
        session.read_transaction(get_sequence, rv)
    return rv

In [ ]:
def was_parameter_names_keys_from_experiment_and_procedure_names(ex_name, proc_name):
    q = """
MATCH (e:Experiment {name: $ex_name})
-[:INCLUDES]->(proc:Procedure {name: $proc_name})
-[:INCORPORATES]->(par:Parameters)
RETURN par.name as name, par.unikey as key
"""
    return [(r['name'], r['key']) for r in 
            nj.query_read_yield(driver, q, ex_name=ex_name, proc_name=proc_name)]

In [ ]:
def data_from_run(procedure_unikey):
    q = """
MATCH (par:Parameters  {unikey: $key})-[:configures]->(head:Net)
MATCH p=(head)-[:LEARNED*]->(tail:Net)
WHERE NOT (tail)-[:LEARNED]->()
RETURN 
    head.shorthand as shorthand,
    [x IN nodes(p) | x.ksv] as ksvs,
    [x IN nodes(p) | x.loss] as losses
"""
    r = nj.query_read_return_list(driver, q, key=procedure_unikey)
    return dict(r[0])

In [ ]:
#data_from_run('MSBTDgD7m87O3xMIV3FIXA')

In [ ]:
def data_from_experiment_and_procedure_names(ex_name, proc_name):
    q = """
MATCH (e:Experiment {name: $ex_name})
-[:INCLUDES]->(proc:Procedure {name: $proc_name})
-[:INCORPORATES]->(par:Parameters)
-[:CONFIGURES]->(head:Net)
MATCH p=(head)-[:LEARNED*]->(tail:Net)
WHERE NOT (tail)-[:LEARNED]->()
RETURN 
    par.name as name,
    head.shorthand as shorthand,
    [x IN nodes(p) | x.ksv] as ksvs,
    [x IN nodes(p) | x.loss] as losses
ORDER BY name
"""
    return [Thing(**dict(r)) for r in nj.query_read_yield(driver, q, ex_name=ex_name, proc_name=proc_name)]

In [ ]:
def data_from_parameter_keys(key_list):
    q = """
MATCH (par:Parameters)
-[:CONFIGURES]->(head:Net)
WHERE par.unikey IN $key_list
MATCH p=(head)-[:LEARNED*]->(tail:Net)
WHERE NOT (tail)-[:LEARNED]->()
RETURN 
    par.name as name,
    head.shorthand as shorthand,
    [x IN nodes(p) | x.ksv] as ksvs,
    [x IN nodes(p) | x.loss] as losses
ORDER BY name
"""
    return [Thing(**dict(r)) for r in nj.query_read_yield(driver, q, key_list=key_list)]

In [ ]:
data_from_parameter_keys(['C2dHgYX4OgdDzpauvNbnIA', 'CRaYV7K8fmyoCALM9n33VQ'])

In [ ]:
def ix_or_default(seq, ix, default):
    try:
        return seq[ix]
    except IndexError:
        return default

In [ ]:
def show_adcs_n(n):
    for nrp, thing in zip(nrps, things):
        nrp(sv_from_ksv(ix_or_default(thing.ksvs, n, thing.ksvs[-1])))
    return [ix_or_default(thing.losses, n, 0) for thing in things]

## UI

In [ ]:
exp_w = widgets.Dropdown(
    #options=[('One', 1), ('Two', 2), ('Three', 3)],
    options=sorted(gu.get_experiment_names_keys(driver)),
    #value=2,
    description='Experiment:',
    disabled=False,
)

In [ ]:
proc_w = widgets.Dropdown(
    options = sorted(gu.get_procedure_names_keys_from_experiment_key(driver, exp_w.value)),
    description='Procedure:',
    disabled=False,
)

In [ ]:
params_w = widgets.SelectMultiple(
    #options=[('Apples', 1), ('Bananas', 2), ('Cherries', 3), ('Dates', 4)],
    #value=(1, 3),
    options=sorted(gu.parameter_names_keys_from_experiment_and_procedure_keys(driver, exp_w.value, proc_w.value)),
    rows=8,
    description='Parameters',
    disabled=False
)

In [ ]:
def update_things(v):
    global things
    things = data_from_parameter_keys(list(v))

In [ ]:
cf(update_things)

In [ ]:
%debug